# Georeferences videos

Input files are a video (\*.mp4) recorded using the camera app and a log file (\*.csv) created using the GPSLogger app.

Output file is a csv file containing frame number, UTC timestamp, latitude and longitude.

In [1]:
import cv2
import pandas as pd
import numpy as np
import exiftool
from datetime import datetime, timedelta
import pytz
import logging

In [2]:
def get_video_start_stop(filename):
    """
    """  
    with exiftool.ExifTool() as et:
        createDate = et.get_tag('QuickTime:CreateDate', filename)
        logging.info(f'Exif Quicktime:CreateDate: {createDate} UTC')
        duration = et.get_tag('QuickTime:Duration', filename)
        logging.info(f'Exif QuickTime:Duration: {duration} seconds')
    createDate = datetime.strptime(createDate, '%Y:%m:%d %H:%M:%S')   
    start = createDate - timedelta(seconds=duration)
    logging.info(f'Video started at {start} UTC')
    stop = createDate
    logging.info(f'Video ended at {stop} UTC')
    return start, stop

#get_video_start_stop(filename)

In [3]:
def get_lat_lon(timestamp):
    timestamp = pd.to_datetime(timestamp)
    df = dfgps[dfgps.time==timestamp]
    if not df.empty:
        # There is a record for exact timestamp (unlikely); return lat lon.
        return dfgps.lat.values[0], dfgps.lon.values[0]
    else:
        # Estimate lat lon using linear interpolation records just prior and post timestamp 
        df1 = dfgps[dfgps.time<timestamp].tail(1)     
        df2 = dfgps[dfgps.time>timestamp].head(1)
        t1 = df1.time.values[0]
        t2 = df2.time.values[0]
        lat1 = df1.lat.values[0]
        lat2 = df2.lat.values[0]
        lon1 = df1.lon.values[0]
        lon2 = df2.lon.values[0] 
        fraction = (timestamp-t1)/(t2-t1)
        lat = lat1 + fraction*(lat2-lat1)
        lon = lon1 + fraction*(lon2-lon1)
        return lat, lon

#start, stop = get_video_start_stop(filename)    
#get_lat_lon(start)

In [4]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(funcName)s %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%S%z",
    handlers=[
        logging.FileHandler("debug.log"),
        logging.StreamHandler(),
    ]
)

logging.info('Reading videos.csv')
dfvideos = pd.read_csv('videos.csv')

for i, r in dfvideos.iterrows():
    videopath =  r.videopath
    gpslogpath = r.gpslogpath
    frameinterval = r.frameinterval
    
    logging.info(f'Started processing {videopath}')
    logging.info(f'Reading GPS log from {gpslogpath}')
    dfgps = pd.read_csv(gpslogpath, parse_dates=['time'])
    dfgps['time'] = dfgps['time'].dt.tz_localize(None)

    logging.info('Building timestamp-location table.')
    start_time, stop_time = get_video_start_stop(videopath)
    data = list()
    cap = cv2.VideoCapture(videopath)
    i = 0
    while(cap.isOpened()):
        frame_exists, curr_frame = cap.read()
        if frame_exists:
            pos_msec = cap.get(cv2.CAP_PROP_POS_MSEC)
            timestamp = start_time + timedelta(milliseconds=pos_msec)
            lat, lon = get_lat_lon(timestamp)
            data.append([i, timestamp, lat, lon])
            i += 1
            if (i%frameinterval==0):
                logging.info(f'{i} {timestamp} {lat:7.4f} {lon:8.4f}')
        else:
            break
    cap.release()
    dfts = pd.DataFrame(data, columns=['frame', 'timestamp', 'lat', 'lon'])
    outputpath = videopath.replace('.mp4','_gps.csv')
    logging.info(f'Saving output to {outputpath}')                               
    dfts.to_csv(outputpath, index=False)
    logging.info(f'Finished processing {videopath}') 
logging.info('FINISED ALL')    

2020-06-28T10:28:36+1000 [INFO] <module> Reading videos.csv


2020-06-28T10:28:36+1000 [INFO] <module> Started processing /home/aubrey/Desktop/2020-06-19/20200623/20200623_140136.mp4


2020-06-28T10:28:36+1000 [INFO] <module> Reading GPS log from /home/aubrey/Desktop/2020-06-19/20200623/20200623.csv


2020-06-28T10:28:36+1000 [INFO] <module> Building timestamp-location table.


2020-06-28T10:28:36+1000 [INFO] get_video_start_stop Exif Quicktime:CreateDate: 2020:06:23 04:11:00 UTC


...

2020-06-28T10:30:47+1000 [INFO] <module> 14400 2020-06-23 04:09:37.216778 13.4303 144.8036


2020-06-28T10:31:04+1000 [INFO] <module> 16200 2020-06-23 04:10:37.304344 13.4332 144.8043


2020-06-28T10:31:10+1000 [INFO] <module> Saving output to /home/aubrey/Desktop/2020-06-19/20200623/20200623_140136_gps.csv


2020-06-28T10:31:10+1000 [INFO] <module> Finished processing /home/aubrey/Desktop/2020-06-19/20200623/20200623_140136.mp4


2020-06-28T10:31:10+1000 [INFO] <module> FINISED ALL
